In [32]:
import os
import typing

# Import your dependencies
import joblib
import pandas as pd
import scipy
import sklearn.metrics
import warnings
from arch.univariate.base import ConvergenceWarning
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings("ignore", category=DataConversionWarning)


X_train = pd.read_parquet("X_train.parquet")
y_train = pd.read_parquet("y_train.parquet")

In [64]:
import numpy as np
import pandas as pd
from scipy import stats
from multiprocessing import Pool, cpu_count
import numpy as np, pandas as pd
from scipy import stats
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline
from scipy.stats import median_abs_deviation
from scipy.stats import wasserstein_distance
from arch import arch_model  
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.stats.diagnostic import acorr_ljungbox
import lightgbm as lgb
from catboost import CatBoostClassifier
from scipy.stats import linregress

import numpy as np

import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import wasserstein_distance
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.stats.diagnostic import acorr_ljungbox
from arch import arch_model
from scipy.stats import linregress
from sklearn.feature_selection import VarianceThreshold

# ==== Utility Functions ====

def rolling_std_features(x, windows=[5,10,20]):
    results = {}
    s = pd.Series(x)
    for w in windows:
        if len(s) >= w:
            roll_std = s.rolling(window=w).std().dropna()
            results[f'rollstd_mean_w{w}'] = roll_std.mean()
            results[f'rollstd_std_w{w}'] = roll_std.std()
        else:
            results[f'rollstd_mean_w{w}'] = np.nan
            results[f'rollstd_std_w{w}'] = np.nan
    return results

def symmetric_kl(p, q, epsilon=1e-10):
    p = p + epsilon
    q = q + epsilon
    p /= p.sum()
    q /= q.sum()
    kl_pq = np.sum(p * np.log(p / q))
    kl_qp = np.sum(q * np.log(q / p))
    return 0.5 * (kl_pq + kl_qp)

def estimate_garch_volatility(x):
    if len(x) < 30:
        return np.nan
    try:
        am = arch_model(x, vol='Garch', p=1, q=1, dist='normal', rescale=False)
        res = am.fit(disp='off')
        cond_vol = res.conditional_volatility
        return np.mean(cond_vol)
    except Exception:
        return np.nan

def autocorr_features(x, max_lag=5):
    results = {}
    if len(x) < max_lag + 1:
        for lag in range(1, max_lag + 1):
            results[f'acf_lag{lag}'] = np.nan
            results[f'pacf_lag{lag}'] = np.nan
        return results
    try:
        acf_vals = acf(x, nlags=max_lag, fft=False, missing='drop')
        pacf_vals = pacf(x, nlags=max_lag, method='ywunbiased')
        for lag in range(1, max_lag + 1):
            results[f'acf_lag{lag}'] = acf_vals[lag]
            results[f'pacf_lag{lag}'] = pacf_vals[lag]
    except Exception:
        for lag in range(1, max_lag + 1):
            results[f'acf_lag{lag}'] = np.nan
            results[f'pacf_lag{lag}'] = np.nan
    return results

def ljung_box_pvalues(x, max_lag=5):
    results = {}
    if len(x) < max_lag + 1:
        for lag in range(1, max_lag + 1):
            results[f'lb_pvalue_lag{lag}'] = np.nan
        return results
    try:
        _, pvals = acorr_ljungbox(x, lags=max_lag, return_df=False)
        for lag in range(1, max_lag + 1):
            results[f'lb_pvalue_lag{lag}'] = pvals[lag - 1]
    except Exception:
        for lag in range(1, max_lag + 1):
            results[f'lb_pvalue_lag{lag}'] = np.nan
    return results

def linear_trend_slope(x):
    if len(x) < 2:
        return np.nan
    try:
        t = np.arange(len(x))
        slope, _, _, _, _ = linregress(t, x)
        return slope
    except Exception:
        return np.nan

def hurst_exponent(ts):
    N = len(ts)
    if N < 20:
        return np.nan
    ts = np.array(ts)
    mean_ts = np.mean(ts)
    Y = np.cumsum(ts - mean_ts)
    R = np.max(Y) - np.min(Y)
    S = np.std(ts, ddof=1)
    if S == 0:
        return np.nan
    return np.log(R / S) / np.log(N)

def higuchi_fd(x, kmax=10):
    x = np.asarray(x)
    N = len(x)
    L = []
    for k in range(1, kmax + 1):
        Lk = []
        for m in range(k):
            length = 0
            count = 0
            for i in range(1, int(np.floor((N - m) / k))):
                length += abs(x[m + i * k] - x[m + (i - 1) * k])
                count += 1
            if count > 0:
                norm_length = (length * (N - 1) / (count * k))
                Lk.append(norm_length)
        if len(Lk) > 0:
            L.append(np.mean(Lk))
    if len(L) < 2:
        return np.nan
    lnL = np.log(L)
    lnk = np.log(1. / np.arange(1, kmax + 1))
    coeffs = np.polyfit(lnk, lnL, 1)
    return coeffs[0]

def petrosian_fd(x):
    x = np.asarray(x)
    N = len(x)
    if N < 2:
        return np.nan
    diff = np.diff(x)
    sign_changes = np.sum(diff[1:] * diff[:-1] < 0)
    if sign_changes == 0:
        return 0.0
    return np.log10(N) / (np.log10(N) + np.log10(N / (N + 0.4 * sign_changes)))

def dominant_frequency(x):
    n = len(x)
    if n < 10:
        return np.nan
    x = x - np.mean(x)
    fft_vals = np.fft.rfft(x)
    fft_freqs = np.fft.rfftfreq(n)
    magnitudes = np.abs(fft_vals)
    magnitudes[0] = 0
    if np.all(magnitudes == 0):
        return 0.0
    dom_freq = fft_freqs[np.argmax(magnitudes)]
    return dom_freq

def count_local_extrema(x):
    x = np.asarray(x)
    if len(x) < 3:
        return 0
    maxima = ((x[1:-1] > x[:-2]) & (x[1:-1] > x[2:])).sum()
    minima = ((x[1:-1] < x[:-2]) & (x[1:-1] < x[2:])).sum()
    return maxima + minima

def proportion_identical_consecutive(x):
    if len(x) < 2:
        return np.nan
    x = np.asarray(x)
    return np.mean(x[1:] == x[:-1])

def average_run_length(x):
    if len(x) == 0:
        return np.nan
    x = np.asarray(x)
    run_lengths = []
    current_run_length = 1
    for i in range(1, len(x)):
        if x[i] == x[i-1]:
            current_run_length += 1
        else:
            run_lengths.append(current_run_length)
            current_run_length = 1
    run_lengths.append(current_run_length)
    return np.mean(run_lengths)

def welch_t_statistic(x1, x2):
    try:
        t_stat, p_value = stats.ttest_ind(x1, x2, equal_var=False, nan_policy='omit')
    except Exception:
        t_stat, p_value = np.nan, np.nan
    return t_stat, p_value

# ==== Main feature extraction ====

def extract_features(df, n_bins=50, max_lag=5):
    feats = []
    ids = df.index.get_level_values(0).unique()
    
    for i in ids:
        s = df.loc[i]
        before = s[s['period'] == 0]['value'].values
        after = s[s['period'] == 1]['value'].values
        
        # Basic aggregate stats
        def agg(x):
            if len(x) == 0:
                return dict(mean=np.nan, std=np.nan, median=np.nan,
                            skew=np.nan, kurt=np.nan,
                            q10=np.nan, q25=np.nan, q75=np.nan, q90=np.nan,
                            outliers=np.nan)
            return dict(
                mean=np.mean(x),
                std=np.std(x, ddof=1),
                median=np.median(x),
                skew=stats.skew(x),
                kurt=stats.kurtosis(x),
                q10=np.percentile(x, 10),
                q25=np.percentile(x, 25),
                q75=np.percentile(x, 75),
                q90=np.percentile(x, 90),
                outliers=((np.abs(x - np.mean(x)) > 3 * np.std(x)).sum()) / len(x)
            )
        
        b = agg(before)
        a = agg(after)
        
        # Rolling std features
        b_rollstd = rolling_std_features(before)
        a_rollstd = rolling_std_features(after)
        
        # Distribution distances
        try:
            combined = np.concatenate([before, after])
            bins = np.histogram_bin_edges(combined, bins=n_bins)
            p_hist, _ = np.histogram(before, bins=bins, density=True)
            q_hist, _ = np.histogram(after, bins=bins, density=True)
            kl_div = symmetric_kl(p_hist, q_hist)
        except Exception:
            kl_div = np.nan
        
        try:
            wdist = wasserstein_distance(before, after)
        except Exception:
            wdist = np.nan
        
        # Autocorr & pacf
        b_acf = autocorr_features(before, max_lag)
        a_acf = autocorr_features(after, max_lag)
        
        # Ljung-Box
        b_lb = ljung_box_pvalues(before, max_lag)
        a_lb = ljung_box_pvalues(after, max_lag)
        
        # GARCH vol
        b_garch_vol = estimate_garch_volatility(before)
        a_garch_vol = estimate_garch_volatility(after)
        
        # Linear trend slope
        b_slope = linear_trend_slope(before)
        a_slope = linear_trend_slope(after)
        
        # Hurst exponent
        b_hurst = hurst_exponent(before)
        a_hurst = hurst_exponent(after)
        
        # Fractal dimensions
        b_hfd = higuchi_fd(before)
        a_hfd = higuchi_fd(after)
        b_pfd = petrosian_fd(before)
        a_pfd = petrosian_fd(after)
        
        # Dominant freq
        b_dom_freq = dominant_frequency(before)
        a_dom_freq = dominant_frequency(after)
        
        # Local extrema count
        b_extrema = count_local_extrema(before)
        a_extrema = count_local_extrema(after)
        
        # Runs & repeats
        b_len = len(before)
        a_len = len(after)
        b_prop_identical = proportion_identical_consecutive(before)
        a_prop_identical = proportion_identical_consecutive(after)
        b_avg_run_length = average_run_length(before)
        a_avg_run_length = average_run_length(after)
        
        # Welch's t-test
        tstat, tp = welch_t_statistic(before, after)
        
        row = {'id': i}
        
        # Add basic stats
        for k, v in b.items(): row[f'b_{k}'] = v
        for k, v in a.items(): row[f'a_{k}'] = v
        
        # Rolling std
        for k, v in b_rollstd.items(): row[f'b_{k}'] = v
        for k, v in a_rollstd.items(): row[f'a_{k}'] = v
        
        # Distribution distances
        row['kl_divergence'] = kl_div
        row['wasserstein'] = wdist
        
        # Autocorr & pacf
        for k, v in b_acf.items(): row[f'b_{k}'] = v
        for k, v in a_acf.items(): row[f'a_{k}'] = v
        
        # Ljung-Box p-values
        for k, v in b_lb.items(): row[f'b_{k}'] = v
        for k, v in a_lb.items(): row[f'a_{k}'] = v
        
        # GARCH volatility
        row['b_garch_vol'] = b_garch_vol
        row['a_garch_vol'] = a_garch_vol
        
        # Trend slope
        row['b_slope'] = b_slope
        row['a_slope'] = a_slope
        
        # Hurst exponent
        row['b_hurst'] = b_hurst
        row['a_hurst'] = a_hurst
        
        # Fractal dimensions
        row['b_hfd'] = b_hfd
        row['a_hfd'] = a_hfd
        row['b_pfd'] = b_pfd
        row['a_pfd'] = a_pfd
        
        # Dominant frequency
        row['b_dom_freq'] = b_dom_freq
        row['a_dom_freq'] = a_dom_freq
        
        # Local extrema count
        row['b_local_extrema'] = b_extrema
        row['a_local_extrema'] = a_extrema
        
        # Runs & repeats
        row['b_length'] = b_len
        row['a_length'] = a_len
        row['b_prop_identical'] = b_prop_identical
        row['a_prop_identical'] = a_prop_identical
        row['b_avg_run_length'] = b_avg_run_length
        row['a_avg_run_length'] = a_avg_run_length
        
        # Welch t-test
        row['welch_tstat'] = tstat
        row['welch_tp'] = tp
        
        # Differences
        row['mean_diff'] = b['mean'] - a['mean']
        row['std_ratio'] = a['std'] / (b['std'] + 1e-9)
        row['kl_div_diff'] = kl_div  # single dist metric, no direct diff needed
        row['wasserstein_diff'] = wdist
        row['garch_vol_diff'] = b_garch_vol - a_garch_vol
        row['slope_diff'] = b_slope - a_slope
        row['hurst_diff'] = b_hurst - a_hurst
        row['hfd_diff'] = b_hfd - a_hfd
        row['pfd_diff'] = b_pfd - a_pfd
        row['dom_freq_diff'] = b_dom_freq - a_dom_freq
        row['local_extrema_diff'] = b_extrema - a_extrema
        row['length_diff'] = b_len - a_len
        row['prop_identical_diff'] = b_prop_identical - a_prop_identical
        row['avg_run_length_diff'] = b_avg_run_length - a_avg_run_length
        
        feats.append(row)
    
    return pd.DataFrame(feats).set_index('id')

def extract_and_select_features(df, y, top_n=30):
    """
    Extract features from df, select top_n features using CatBoost importance.
    
    Parameters:
        df: DataFrame with MultiIndex (id,time) and columns ['value', 'period']
        y: pandas Series with index = ids, values = target bool (structural break)
        top_n: number of features to select
    
    Returns:
        X_selected: DataFrame of selected features indexed by id
        model: trained CatBoostClassifier on selected features
        selected_features: list of feature names selected
    """
    print("Extracting features...")
    X = extract_features(df)
    y = y.loc[X.index].astype(int)
    
    print("Aligning target and features...")
    X = X.loc[y.index].copy()
    
    print("Filling missing values...")
    X = X.fillna(X.median())
    
    print("Removing near-constant features...")
    selector = VarianceThreshold(threshold=1e-5)
    selector.fit(X)
    X_sel = X.loc[:, selector.get_support()]
    print(f"Features remaining after variance threshold: {X_sel.shape[1]}")
    
    print("Training CatBoost for feature importance...")
    model = CatBoostClassifier(verbose=0, random_seed=42)
    model.fit(X_sel, y)
    
    importances = model.get_feature_importance()
    feat_importances = pd.Series(importances, index=X_sel.columns).sort_values(ascending=False)
    
    selected_features = feat_importances.head(top_n).index.tolist()
    X_selected = X_sel[selected_features]
    
    print(f"Selected top {top_n} features:")
    print(selected_features)
    
    print("Retraining CatBoost on selected features...")
    model_final = CatBoostClassifier(verbose=0, random_seed=42)
    model_final.fit(X_selected, y)
    
    return X_selected, model_final, selected_features

# Assuming df is your time series DataFrame and y your labels

X_selected, trained_model, features = extract_and_select_features(X_train.loc[:5000], y_train, top_n=30)

# Now you can use X_selected and trained_model for predictions or further tuning


# X = extract_features(X_train.loc[:5000])


# model = Pipeline([
#     ('scaler', RobustScaler()),          # optional but recommended for robustness
#     ('catboost', CatBoostClassifier(verbose=0))  # silent training
# ])

# cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
# scores = cross_val_score(model, X, y, cv=cv, scoring='roc_auc', n_jobs=-1)

# print(f"CatBoost ROC AUC CV mean: {scores.mean():.4f} ± {scores.std():.4f}")


Extracting features...


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 4. The me

Aligning target and features...
Filling missing values...
Removing near-constant features...
Features remaining after variance threshold: 65
Training CatBoost for feature importance...
Selected top 30 features:
['std_ratio', 'a_skew', 'garch_vol_diff', 'b_kurt', 'a_outliers', 'hfd_diff', 'b_outliers', 'welch_tp', 'b_mean', 'a_kurt', 'b_hfd', 'a_dom_freq', 'kl_div_diff', 'a_lb_pvalue_lag1', 'hurst_diff', 'b_hurst', 'b_skew', 'mean_diff', 'a_hurst', 'welch_tstat', 'kl_divergence', 'b_dom_freq', 'dom_freq_diff', 'b_lb_pvalue_lag1', 'a_length', 'a_hfd', 'a_lb_pvalue_lag4', 'a_q75', 'local_extrema_diff', 'b_length']
Retraining CatBoost on selected features...


In [66]:

y = y_train.loc[X_selected.index].astype(int)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(trained_model, X_selected, y, cv=cv, scoring='roc_auc', n_jobs=-1)

print(f"CatBoost ROC AUC CV mean: {scores.mean():.4f} ± {scores.std():.4f}")

CatBoost ROC AUC CV mean: 0.6653 ± 0.0093


In [67]:
X_selected

,std_ratio,a_skew,garch_vol_diff,b_kurt,a_outliers,hfd_diff,b_outliers,welch_tp,b_mean,a_kurt,...,kl_divergence,b_dom_freq,dom_freq_diff,b_lb_pvalue_lag1,a_length,a_hfd,a_lb_pvalue_lag4,a_q75,local_extrema_diff,b_length
id,,,,,,,,,,,,,,,,,,,,,
0,0.984160,-0.097450,0.000090,0.077383,0.000000,0.026106,0.003701,0.985237,0.000015,-0.273389,...,0.253773,0.298298,0.131631,9.559823e-09,294,1.001470,3.896767e-02,0.004714,721,1351
1,0.806393,-1.019894,-0.053414,6.366972,0.007092,-0.019402,0.014241,0.099826,0.000128,7.915152,...,0.545743,0.211838,-0.114403,2.051478e-02,282,1.010145,3.611862e-02,0.001103,1294,2247
2,1.329722,0.562009,-0.004736,2.699075,0.017476,0.004955,0.014574,0.191140,0.000389,4.326094,...,0.268479,0.108873,-0.269768,2.182911e-07,515,1.009657,1.949231e-05,0.014051,1247,2333
3,1.107024,0.620343,-0.000654,3.751807,0.011129,0.017337,0.010568,0.893657,0.000381,3.215873,...,0.240785,0.148393,0.107057,5.027546e-01,629,0.975371,5.209222e-01,0.005539,1062,2271
4,1.028507,0.047834,-0.000090,-0.127050,0.000000,0.005839,0.000604,0.824822,-0.000016,-0.111458,...,0.235785,0.022933,0.016354,3.685863e-104,456,0.828421,6.250540e-52,0.002235,708,1657
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4996,0.759490,-0.026722,0.002367,1.714886,0.008759,0.013926,0.008614,0.206642,0.000604,1.027076,...,0.242487,0.235029,0.163496,1.162043e-03,685,0.983000,8.679410e-01,0.005485,1208,2438
4997,0.506304,-2.296193,0.001654,3.029120,0.011553,0.026960,0.017118,0.563791,0.000117,27.254992,...,1.102171,0.128106,-0.313486,1.559798e-02,779,0.971573,3.458812e-05,0.000889,662,1811
4998,0.806379,1.239423,0.002390,4.049025,0.013289,0.012922,0.012745,0.759381,0.000768,11.668970,...,0.416482,0.481373,0.062768,4.373863e-01,602,0.996986,7.507826e-01,0.005859,325,1020
